# Homework 3: Language Modelling in Hangman

Student Name:Wai Yan Wong

Student ID:892083

## General info

<b>Due date</b>:  Friday, 17 May 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Overview

## The Hangman Game (*No implementation is needed*)

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. 

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided gusser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('Enter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to True. When submitting your solution, set to False so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [ ]:
if interactive:
    hangman('whatever', human, 8, True)


## 1. Preparing Test Set and Training Set (1 mark)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the algorithm. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown` and the `words` method, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(1 mark)

In [4]:
from nltk.corpus import brown
import numpy as np

np.random.seed(12345)

# word_set stores all the unique word types in the Brown corpus
word_set = []
# test_set stores 1000 word types for testing
test_set = []
# training_set stores the rest word types for training
training_set = []

###
# Your answer BEGINS HERE
###

for word in brown.words():
    if word.isalpha():
        wordlow = word.lower()
        if wordlow not in word_set:
            word_set.append(wordlow)  

np.random.shuffle(word_set)
test_set = word_set[:1000]
training_set = word_set[1000:]


###
# Your answer ENDS HERE
###

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


<b>For your testing:</b>

In [5]:
assert(len(word_set) > 35000 and len(word_set) < 45000)

In [6]:
assert(len(test_set) == 1000)

In [7]:
assert(len(training_set) + len(test_set) == len(word_set))

In [ ]:
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

## 2. Simple Guesser: Random Guessing (1 mark)

<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

(1 mark)

In [5]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [6]:
import string

def random_guesser(mask, guessed, **kwargs):
    """
        This function implements a random guesser. It returns the random guess. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    choose_range = []
    for i in range(26):
        alp = chr(i+ord('a'))
        if alp not in guessed:
            choose_range.append(alp)
    return np.random.choice(choose_range)
    
    ###
    # Your answer ENDS HERE
    ###

# uncomment to run a single hangman game with output shown (useful for debugging)
hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("Average number of incorrect guesses: ", result)

Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ _ _ _ length 14
You have 10 attempts remaining.
Guess is r
Good guess: _ _ _ r _ _ _ _ _ _ _ _ _ _
You have 10 attempts remaining.
Guess is x
Sorry, try again.
You have 9 attempts remaining.
Guess is l
Good guess: _ _ _ r _ _ _ _ _ _ _ _ l _
You have 9 attempts remaining.
Guess is m
Sorry, try again.
You have 8 attempts remaining.
Guess is h
Sorry, try again.
You have 7 attempts remaining.
Guess is u
Sorry, try again.
You have 6 attempts remaining.
Guess is k
Sorry, try again.
You have 5 attempts remaining.
Guess is z
Sorry, try again.
You have 4 attempts remaining.
Guess is d
Sorry, try again.
You have 3 attempts remaining.
Guess is c
Good guess: _ _ _ r _ c _ _ _ _ _ _ l _
You have 3 attempts remaining.
Guess is w
Sorry, try again.
You have 2 attempts remaining.
Guess is v
Good guess: _ _ _ r _ c _ _ _ _ v _ l _
You have 2 attempts remaining.
Guess is q
Sorry, try again.
You have 1 attempts remaining.
Guess is p
Good guess: _ p p

<b>For your testing:</b>

In [7]:
assert(result > 10 and result < 20)

## 3. Your First AI Guesser: Unigram Guesser (1 mark)

**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

Hint: It should be much lower than random guessing.

(1 mark)

In [8]:
from collections import Counter

# unigram_counts stores the frequencies of characters over all training words
unigram_counts = Counter()

###
# Your answer BEGINS HERE
###

for word in training_set:
    for i in range(len(word)):
        unigram_counts[word[i]] += 1

###
# Your answer ENDS HERE
###

print(unigram_counts)

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        This function implements a unigram guesser. It returns a guess based on the unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###

    choose_range = []
    for i in range(26):
        alp = chr(i+ord('a'))
        if alp not in guessed:
            choose_range.append(alp)
            
    total = float(sum(unigram_counts.values()))
    alp_pro = {}
    for alp in choose_range:
        alp_pro[alp] = unigram_counts[alp]/total  
        
    if alp_pro:
        return max(alp_pro,key=alp_pro.get)
    else:
        return random_guesser(mask, guessed)

    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("Average number of incorrect guesses: ", result)

Counter({'e': 35195, 'i': 26042, 'a': 24344, 's': 23736, 'n': 22553, 'r': 22113, 't': 20658, 'o': 19144, 'l': 16678, 'c': 12615, 'd': 12249, 'u': 10198, 'm': 8687, 'g': 8604, 'p': 8559, 'h': 7345, 'b': 5743, 'y': 5353, 'f': 4220, 'v': 3439, 'k': 2983, 'w': 2786, 'z': 1026, 'x': 829, 'j': 639, 'q': 535})

Average number of incorrect guesses:  10.308


<b>For your testing:</b>

In [9]:
assert(result > 5 and result < 15)

## 4. Your Second AI Guesser: Length-based Unigram Guesser (1 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having a *different* unigram model for each length of the words. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. You need to decide on how to handle this situation.

(1 mark)

In [10]:
from collections import defaultdict

# unigram_counts_by_length stores a dictionary, mapping word length to the frequencies of characters of words with that word length
unigram_counts_by_length = defaultdict(Counter)

###
# Your answer BEGINS HERE
###

for word in training_set:
    counts = unigram_counts_by_length[len(word)]
    for i in range(len(word)):
        counts[word[i]] += 1
    unigram_counts_by_length[len(word)] = counts

###
# Your answer ENDS HERE
###

lengths = sorted(unigram_counts_by_length.keys())
max_length = lengths[-1] + 1
print(lengths)
print()
print(unigram_counts_by_length)

def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    """
        This function implements a length-based unigram guesser. It returns a guess based on the length-based unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    choose_range = []
    for i in range(26):
        alp = chr(i+ord('a'))
        if alp not in guessed:
            choose_range.append(alp)
    
    if len(mask) not in lengths:
        return unigram_guesser(mask, guessed, unigram_counts=unigram_counts)
    else:
        current_counts = counts[len(mask)]
        
    total = float(sum(current_counts.values()))
    alp_pro = {}
    for alp in choose_range:
        alp_pro[alp] = current_counts[alp]/total
    
    if alp_pro:
        return max(alp_pro,key=alp_pro.get)
    else:
        return unigram_guesser(mask, guessed, unigram_counts=unigram_counts)
    
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("Average number of incorrect guesses: ", result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

defaultdict(<class 'collections.Counter'>, {6: Counter({'e': 4246, 'a': 2672, 's': 2598, 'r': 2537, 'i': 2038, 'n': 1984, 'o': 1969, 'l': 1948, 't': 1849, 'd': 1630, 'u': 1147, 'c': 1066, 'm': 937, 'g': 883, 'h': 870, 'p': 853, 'b': 780, 'y': 705, 'f': 497, 'k': 452, 'w': 426, 'v': 391, 'z': 123, 'j': 102, 'x': 91, 'q': 50}), 5: Counter({'e': 2220, 's': 1843, 'a': 1833, 'r': 1371, 'o': 1332, 'l': 1215, 'i': 1157, 't': 1097, 'n': 1034, 'd': 764, 'c': 705, 'u': 680, 'h': 589, 'm': 577, 'y': 542, 'p': 535, 'b': 507, 'g': 465, 'k': 382, 'f': 342, 'w': 300, 'v': 238, 'z': 86, 'j': 77, 'x': 64, 'q': 30}), 7: Counter({'e': 5320, 'a': 3568, 's': 3462, 'r': 3433, 'i': 3340, 'n': 3036, 't': 2664, 'o': 2508, 'l': 2491, 'd': 2061, 'c': 1680, 'u': 1508, 'g': 1490, 'm': 1241, 'p': 1237, 'h': 1097, 'b': 945, 'y': 749, 'f': 675, 'k': 582, 'w': 517, 'v': 462, 'z': 156, 'x': 112, 'j': 96, 'q': 83}), 13: Counter({'i': 1283, 

<b>For your testing:</b>

In [11]:
assert(result > 5 and result < 15)

## 5. Your Third AI Guesser: Bigram Guesser (1 mark)

**Instructions:** Now for the next challenge, using a *bigram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

You should develop a *bigram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with a sentinel symbol `$`.) You should use *linear interpolation* to smooth between the higher order and lower order models, and you will have to decide how to weight each component (be reminded that all probabilities should sum to 1).

Your bigram guesser should apply your language model to each blank position in the secret word by using the left context as is known. E.g., in the partial word `$ _ e c _ e _ _` we know the left context for the first three blanks, but have no known left context for the last blank. Using a bigram language model, we are able to apply it to the first three blanks only. You should then select the character with the highest probability of predicting the most number of correct entries over the blanks. 

Do you see any improvement over the unigram guessers above?

(1 mark)

In [14]:
###
# Your answer BEGINS HERE
###

bigram_counts = None # you will want a different data structure to store the bigram 
bigram_counts = defaultdict(Counter)
# key of the dict is the pre-letter
# calculate the probility for all letters
for word in training_set:
    newword = '$'+word
    for i in range(len(word)):
        bigram_counts[newword[i]][word[i]] += 1
        
bigram_lambda = 0.8
unigram_lambda = 1 - bigram_lambda
uni_total = float(sum(unigram_counts.values()))

bigram_prob = defaultdict(Counter)
for pre_alp in bigram_counts:
    if pre_alp == '$': 
        bi_total = float(sum(bigram_counts[pre_alp].values()))
    else: bi_total = float(unigram_counts[pre_alp])
                    
    for alp in bigram_counts[pre_alp]:
        prob_bigram = bigram_counts[pre_alp][alp]/bi_total*bigram_lambda
        prob_unigram = unigram_lambda*unigram_counts[alp]/uni_total
        bigram_prob[pre_alp][alp] = prob_bigram+prob_unigram

###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, counts=bigram_counts): # add extra default arguments if needed
    """
        This function implements a bigram guesser. It returns a guess based on the bigram model using linear interpolation.
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    mask.insert(0,'$') 
    next_prob = Counter()
    for i in range(1,len(mask)):
        cur_alp = mask[i]
        pre_alp = mask[i-1]
        if cur_alp == '_':
            if pre_alp != '_':
                prob_cur_alp = Counter()
                for key,value in bigram_prob[pre_alp].most_common(len(bigram_prob[pre_alp])):
                    # start the loop in decreasing order of prob
                    # keep checking until the key is not in guessed
                    if key not in guessed:
                        prob_cur_alp[key] = value
                        break
                        
                # compare the highest probs among all the eligible blanks and choose the key with highest prob
                if prob_cur_alp:
                    key = max(prob_cur_alp, key=prob_cur_alp.get)
                    next_prob[key] = max(prob_cur_alp[key],next_prob[key])
                
    mask.remove('$')
    if next_prob:
        return(max(next_prob, key=next_prob.get))
    else:
        return unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length)
      
    ###
    # Your answer ENDS HERE
    ###

#hangman(np.random.choice(test_set), bigram_guesser, 10, True)

result = test_guesser(bigram_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  9.95


<b>For your testing:</b>

In [16]:
assert(result < 13)

## 6. Your Own AI Guesser (1 mark)

**Instructions:** You should try to develop a more effective AI, `my_ai_guesser`, for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, and using ngram models. Ensure you report the test performance of your guesser. Have fun!

You will be marked based on the explanation of your approach and its accuracy. 

(1 mark) 

In [17]:
###
# Your answer BEGINS HERE
###

# calculate length-based bigram probilities
# and length-based unigram probilities
bigram_counts_by_length = defaultdict(lambda: defaultdict(Counter))

for word in training_set:
    cword_dict = bigram_counts_by_length[len(word)]
    newword = '$'+word
    for i in range(len(word)):
        cword_dict[newword[i]][word[i]] += 1
    bigram_counts_by_length[len(word)] = cword_dict


bi_len_single_count = defaultdict(Counter)
for wordlength in bigram_counts_by_length.keys():
    for pre_alp in bigram_counts_by_length[wordlength]:
        for alp in bigram_counts_by_length[wordlength][pre_alp]:
            bi_len_single_count[wordlength][alp] += 1;
            
bigram_length_prob = defaultdict(lambda: defaultdict(Counter))      
    
for wordlength in bigram_counts_by_length.keys():
    for pre_alp in bigram_counts_by_length[wordlength]:
        if pre_alp == '$':  bi_total = float(sum(bigram_counts_by_length[wordlength][pre_alp].values()))
        else: bi_total = float(bi_len_single_count[wordlength][pre_alp])
                    
        for alp in bigram_counts_by_length[wordlength][pre_alp]:
            prob_bigram = bigram_counts_by_length[wordlength][pre_alp][alp]/bi_total*bigram_lambda
            prob_unigram = unigram_lambda*unigram_counts[alp]/uni_total
            bigram_length_prob[wordlength][pre_alp][alp] = prob_bigram+prob_unigram
        
bi_lengths = sorted(bigram_counts_by_length.keys())

unigram_length_prob = defaultdict(Counter)
for length in unigram_counts_by_length:
    for alp in unigram_counts_by_length[length]:
        unigram_length_prob[length][alp] = unigram_counts_by_length[length][alp]/uni_total

        
def my_ai_guesser(mask, guessed, counts=bigram_counts_by_length):
    
    if len(mask) not in bi_lengths:
        return bigram_guesser(mask, guessed, counts=bigram_counts)
    else:
        wordlength = len(mask)
        mask.insert(0,'$') 
        next_prob = Counter()
        for i in range(1,len(mask)):
            cur_alp = mask[i]
            pre_alp = mask[i-1] 
            prob_cur_alp = Counter()
            if cur_alp == '_':
                if pre_alp != '_':
                    corres_prob = bigram_length_prob[wordlength][pre_alp]
                    for alp in corres_prob:
                        if alp not in guessed:
                            # sum the prob of same letters in different blanks
                            next_prob[alp] += corres_prob[alp]
                else:
                    # apply length-based unigram model when the pre-letter is a blank
                    corres_prob = unigram_length_prob[wordlength]
                    for alp in corres_prob:
                        if alp not in guessed:
                            next_prob[alp] += corres_prob[alp]
        
        mask.remove('$')
        if next_prob:
            # choose the letter with highest prob among all letters
            for key,value in next_prob.most_common(len(next_prob)):
                return key
        else:
            return bigram_guesser(mask, guessed, counts=bigram_counts)
                
    
#     if next_prob:
#         return(max(next_prob, key=next_prob.get))
#     else:
#         return bigram_guesser(mask, guessed, counts=bigram_counts)
    

###
# Your answer ENDS HERE
###

result = test_guesser(my_ai_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  8.925


**Instructions:** Explain your approach and discuss your result below. Please keep it brief.